In [1]:
!pip install sentence-splitter

You should consider upgrading via the 'c:\users\waleed\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install transformers

You should consider upgrading via the 'c:\users\waleed\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:

!pip install SentencePiece

You should consider upgrading via the 'c:\users\waleed\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [4]:
#Creating the model here

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=None, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=None,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [5]:
#Passing single sentence here 

text = 'The International Cricket Council is the global governing body of cricket.'

#Getting response, the second parameter defines the number of outputs we want here

get_response(text,3)

c:\Users\Waleed\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['The governing body of cricket is the International Cricket Council.',
 'The global governing body of cricket is the International Cricket Council.',
 'The international governing body of cricket is the International Cricket Council.']

In [6]:
#Checking the model by passing a paragraph

context = 'The International Cricket Council is the global governing body of cricket. It was founded as the Imperial Cricket Conference in 1909 by representatives from Australia, England and South Africa. It was renamed as the International Cricket Conference in 1965, and took up its current name in 1987'
#get_response(context,1)
print(context)

The International Cricket Council is the global governing body of cricket. It was founded as the Imperial Cricket Conference in 1909 by representatives from Australia, England and South Africa. It was renamed as the International Cricket Conference in 1965, and took up its current name in 1987


In [7]:
# Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

['The International Cricket Council is the global governing body of cricket.',
 'It was founded as the Imperial Cricket Conference in 1909 by representatives from Australia, England and South Africa.',
 'It was renamed as the International Cricket Conference in 1965, and took up its current name in 1987']

In [8]:
# Loop through sentences in paragraph and paraphrase each on of them 
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

In [9]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

['The governing body of cricket is the International Cricket Council.',
 'The Imperial Cricket Conference was founded in 1909 by representatives from Australia, England and South Africa.',
 'The International Cricket Conference was changed to its current name in 1987.']

In [10]:
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'The governing body of cricket is the International Cricket Council. The Imperial Cricket Conference was founded in 1909 by representatives from Australia, England and South Africa. The International Cricket Conference was changed to its current name in 1987.'

In [11]:
# Comparison between the real and paraphrased text
print (context)
print(paraphrased_text)

The International Cricket Council is the global governing body of cricket. It was founded as the Imperial Cricket Conference in 1909 by representatives from Australia, England and South Africa. It was renamed as the International Cricket Conference in 1965, and took up its current name in 1987
The governing body of cricket is the International Cricket Council. The Imperial Cricket Conference was founded in 1909 by representatives from Australia, England and South Africa. The International Cricket Conference was changed to its current name in 1987.


In [16]:
#Saving the model
import pickle

with open('pegasus_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
#Code to load the model

with open('pegasus_model.pkl', 'rb') as f:
    model = pickle.load(f)